# Hands on session: Autoencoders (1)
This is the first of two examples, where we will use Autoencoders to reduce the dimensionality of the calcium imaging data. We will use the same dataset as in the previous exercises (PCA, Clustering methods). 

In this example, we encode the calcium data for each neuron, i.e., we represent each neuron in a lower-dimensional space, compressing the time information. 

This exercise refers to Chapter 4 "Autoencoders" of the "Dimensionality reduction in neuroscience" course (tutor: Fabrizio Musacchio, Oct 17, 2024)

## Acknowledgements:
The dataset is from the 2023's course 'data analysis techniques in neuroscience' by the Chen Institute for Neuroscience at Caltech:  

<https://github.com/cheninstitutecaltech/Caltech_DATASAI_Neuroscience_23>

and originally from the paper:

Remedios, R., Kennedy, A., Zelikowsky, M. et al. Social behaviour shapes hypothalamic neural  ensemble representations of conspecific sex. Nature 550, 388–392 (2017). <https://doi.org/10.1038/nature23885>

## Dataset
We will work with the same calcium imaging data from the previous exercise (PCA). For details, please refer to the previous exercise.

## Environment setup
For reproducibility:

```bash
conda create -n dimredcution python=3.11 mamba -y
conda activate dimredcution
mamba install ipykernel matplotlib numpy scipy scikit-learn -y
mamba install pytorch torchvision -c pytorch
```

Let's start, as usual, by importing the necessary libraries:

In [ ]:
# %% IMPORTS
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

from mpl_toolkits import mplot3d
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Subset

# for reproducibility, we set the random seed:
np.random.seed(42)
torch.manual_seed(42)

Verify the torch version and that the GPU is available:

In [ ]:
# verify torch version and GPU availability:
print(f"torch backend MPS is available? {torch.backends.mps.is_available()}")
print(f"current PyTorch installation built with MPS activated? {torch.backends.mps.is_built()}")
print(f"check the torch MPS backend: {torch.device('mps')}")
print(f"test torch tensor on MPS: {torch.tensor([1,2,3], device='mps')}")

On macOS, you need to move your later model to the MPS device, if you want to use the Mac's GPU (Apple Silicon):


In [ ]:
# device = torch.device('mps')
# model = model.to(device)

Otherwise, change this line to:

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)

Define the path to the data:

In [4]:
DATA_PATH = '../data/'
DATA_FILENAME = 'hypothalamus_calcium_imaging_remedios_et_al.mat'
DATA_FILE = os.path.join(DATA_PATH, DATA_FILENAME)

RESULTSPATH = '../results/'
# check whether the results path exists, if not, create it:
if not os.path.exists(RESULTSPATH):
    os.makedirs(RESULTSPATH)

Load the data:

In [ ]:
hypothalamus_data = loadmat(DATA_FILE)

# extract the N main data arrays into N separate variables:
neural_data   = hypothalamus_data['neural_data']
attack_vector = hypothalamus_data['attack_vector']
gender_vector = hypothalamus_data['sex_vector']

# we just take a short snippet of the data to speed up the computations:
N_datapoints = 10000
neural_data_short = neural_data[:, 0:N_datapoints]
print(f"The shape of the data is: {neural_data_short.shape}")

We define a custom dataset class for the neural data (this is necessary for Pytorch's DataLoader):

In [7]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        instance = self.data[idx, :]
        sample = {"data": instance}
        return sample

Pytorch works best with tensors, so we convert the numpy array to a tensor. A tensor is a container that stores data in $N$ dimensions. A matrix is a special case of a tensor that is 2D:

In [8]:
neural_data_float_tensor = torch.tensor(neural_data_short).float()

## 📝 Inspect the shape of the `neural_data_float_tensor`

In [ ]:
# Your code goes here:



Now we apply the `CustomDataset` class to our neural data tensor `neural_data_float_tensor`:

In [10]:
neural_dataset_tensor = CustomDataset(neural_data_float_tensor)

## 📝 Inspect `neural_dataset_tensor` created by the `CustomDataset` class
1. Inspect the `.__len__()` method.
2. Inspect the `.__getitem__(0)` method.
3. Inspect the `.__getitem__(0).keys()` method.
4. Inspect the `.__getitem__(0)['data'].shape` method.
5. Also inspect the above methods for another index.

In [ ]:
# Your code goes here:



Now we are going to break our data into a train (90% of the data) and test (10% of the data) split:

In [ ]:
train_size = int(np.floor(0.9 * len(neural_dataset_tensor)))
test_size = len(neural_dataset_tensor) - train_size

print(train_size, test_size)
# train_size, test_size is (103, 12), ie., 90% (=103/115) and 10% (=12/115) neurons of the full data set

Now using the `torch.utils.data.random_split()` function we split the data into a  train set and a test set; for this, we could use this command:

```python
train_dataset, test_dataset = torch.utils.data.random_split(neural_dataset_tensor, [train_size, test_size])
```

However, we want to keep track of the original neuron IDs, so we will do it manually:

In [17]:
dataset_size = len(neural_dataset_tensor)
indices = list(range(dataset_size))
#train_dataset, test_dataset = torch.utils.data.random_split(neural_dataset_tensor, [train_size, test_size])

# randomly split the indices:
train_indices, test_indices = torch.utils.data.random_split(indices, [train_size, test_size])

# create subsets using the indices:
train_dataset = Subset(neural_dataset_tensor, train_indices)
test_dataset  = Subset(neural_dataset_tensor, test_indices)

# To re-identify the original ID of each neuron in the train_dataset, we store the original indices:
original_train_ids = train_indices.indices

## 📝 Create the data loaders
Using the `torch.utils.data.DataLoader()`, make a train data loader and a test data loader. Make sure to use the dataset, batch_size, and shuffle parameters when you call the function. If you are not familiar with the DataLoader, please check the Pytorch documentation. For simplicity, set the batch size to be larger than all the data you have. This is okay for our small dataset, but in practice, you would want to set the batch size to be a smaller number.

In [18]:
# Your code goes here:

# train_loader = torch.utils.data.DataLoader(...)
# test_loader  = torch.utils.data.DataLoader(...)



## 📝 Now, define the Autoencoder model
1. Create a class `Autoencoder` that inherits from `torch.nn.Module`. Create a linear `encoder` with the following layers:
   - A linear layer with 10000 input features and 5000 output features.
   - A ReLU activation function.
   - A linear layer with 5000 input features and 2500 output features.
   - A ReLU activation function.
   - A linear layer with 2500 input features and 625 output features.
   - A ReLU activation function.
   - A linear layer with 625 input features and 156 output features.
   - A ReLU activation function.
   - A linear layer with 156 input features and 39 output features.
   - A ReLU activation function.
   - A linear layer with 39 input features and 3 output features.
2. Create an according linear `decoder`.
3. Define a `forward` method that takes an input x and returns the embedding and the reconstruction of the input data.

In [19]:
# Your code goes here:
# class Autoencoder(...):
#     def __init__(self):
#         super().__init__()
#        
#         self.encoder = torch.nn.Sequential(
#             ...
#         )
          
#         self.decoder = torch.nn.Sequential(
#             ...
#         )
  
#     def forward(self, x):
#         ...
#         return embedding, reconstruction




## 📝  Train the autoencoder

Prepare and create a training loop to train the autoencoder on the training data:
1. Train the autoencoder for 250 epochs.
2. Keep track of the training and validation losses.
3. As loss function, use the Mean Squared Error (MSE) loss (`torch.nn.MSELoss()`).
4. Use the Adam optimizer (`torch.optim.Adam(...)`) with an initial learning rate of 1e-2, a `weight_decay` of 1e-8 and a `step_size` of 1.
5. Follow further instructions below to complete the code.

In [ ]:
# Your code goes here:

# set training parameters:
# epochs =

# prepare lists to store the outputs and losses:

# model initialization (just uncomment the line below):
# model = Autoencoder()

# on macOS, move the model to the MPS device:
# device = torch.device('mps')
# model = model.to(device)

# otherwise, move the model to the GPU if available:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)


# using Mean-Squared-Error MSE Loss function:
# loss_function = ...

# using an Adam Optimizer:
# learning_rate = ...
# optimizer = torch.optim.Adam(...)

# learning rate scheduler (just uncomment the line below):
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=1, gamma=0.98)


# define the training loop:
#for epoch in range(epochs):
#    # training on train set:
#    model.train()
#    
#    # loop through your training data;
#    for batch_idx, batch in enumerate(train_loader):

        # STEP 1: pull out the data from your batch
        # batch_data = ...
        
        # STEP 2: get the reconstructed data from the Autoencoder Output
        # ...

        # STEP 3: calculate the loss function between the reconstruction and original data
        # loss = ...

        # set gradients to zero (just uncomment the line below):
        #optimizer.zero_grad()
        
        # the gradient is computed and stored (just uncomment the line below):
        #loss.backward()
        
        # perform the parameter update (just uncomment the line below):
        #optimizer.step()

        # storing the losses in a list for plotting
        # ...
       
    # put model into evaluation mode (just uncomment the line below):
    # model.eval()
    
    # loop through your testing/validation data:
    #for validation_batch_idx, validation_batch in enumerate(test_loader):
    
        # STEP_4: pull out the data from your validation batch
        # validation_batch_data = ...
        
        # STEP 5: get the reconstructed data from the Autoencoder Output
        # ...
        
        # STEP 6: calculate the loss function between the reconstrucion and original data
        # val_loss = ...
        
        # STEP 7: append the validation losses to the validation loss list
        #... 
    
    # STEP 8: append the outputs to both the validation and train outputs lists in the form of:
    # (epochs, original data, reconstruction, embedding). Don't forget to transform your tensors into numpy arrays!!!
    # validation_outputs.append(...)
    # outputs.append(...)
    
    # uncomment the line below to print the epoch number and the losses:
    # print(f'Epoch {epoch+1}/{epochs}, Train Loss: {losses[-1]:.6f}, Val Loss: {validation_losses[-1]:.6f}')

#print(len(outputs))

#print(f"epoch number: {outputs[-1][0]}") # epoch number
#print(f"original data: {outputs[-1][-1].shape}") # original data
#print(f"reconstructed data{outputs[-1][-2].shape}") # reconstructed data




## 📝 Assess the loss curves
1. Plot the training and validation loss curves (i.e., your previously stored losses).
2. What do you observe?

In [ ]:
# Your code goes here:



In case you are unsatisfied with the results, you can try to improve the model by 

* changing the architecture: add Dropout layers (`torch.nn.Dropout()`) with a dropout rate of 0.5 to your encoder.
* changing the learning rate; try 1e-3 or 1e-4.

## 📝 For a sample neuron, plot the original and reconstructed data for the last epoch

Hint: plot  
outputs[epoch_idx][1][neuron_idx, :] and  
outputs[epoch_idx][2][neuron_idx, :]

In [ ]:
# Your code goes here:



## 📝 Compare all neurons: original vs. reconstructed
1. Plot the original data and the reconstructed data with Matplotlib's `imshow`
2. Plot the original validation data and the reconstructed validation data with Matplotlib's `imshow`

In [ ]:
# Your code goes here:



## 📝 Perform a PCA in the reconstructed data (2D comparison)
We now want to take a look at the reconstructed on PCA latent space:
1. Perform a PCA on the (original) train data and the reconstructed train data. Use three components.
2. Plot the first two PCs of each PCA (PCA of the original data and the reconstructed data) in a single 2D plot.

In [ ]:
# Your code goes here:



## 📝 Investigate the latent space of the AE (3D)

1. Plot the AE embedding space in 3D at the first epoch (stored in `outputs[0][-1][:, 0]`, `outputs[0][-1][:, 1]`, `outputs[0][-1][:, 2]`)
2. Plot the AE embedding space in 3D at the last epoch (stored in `outputs[-1][-1][:, 0]`, `outputs[-1][-1][:, 1]`, `outputs[-1][-1][:, 2]`). What do you notice by comparing the two plots/epochs?

In [ ]:
# Your code goes here:



## 📝 Replot the AE embedding scatter plot of the last epoch, now color-coded by the original data (`original_train_ids`)

In [ ]:
# Your code goes here:



## 📝 Cluster the AE embedded space of the last epoch using k-means (with 2 clusters) and replot the embedded space color-coded with the cluster labels

In [ ]:
# Your code goes here:



## 📝 Resort the neural data
Let's check whether the clustering is consistent with the original data. To do this, sort the neural data according to the cluster labels and plot the sorted data. Compare with the original data.

In [ ]:
# Your code goes here:

# resort the train_dataset array in such a way, that all neurons of cluster 0 are first, 
# and all neurons of cluster 1 are second:


# now plot the sorted neural data:


In [ ]:
# What do you observe? What can you interpret from it?

